# Validación cruzada


## Autores

1. Alvaro Mauricio Montenegro Díaz, ammontenegrod@unal.edu.co

## Referencias

1. Alvaro Montenegro, [Curso de Estadística Bayesiana](https://github.com/AprendizajeProfundo/Estadistica-Bayesiana), 2020
2. Guangyuan Gao, "Bayesian Claims Reserving Methods in Non-life Insurance with Stan. An Introduction", Springer, 2018

# Introducción

Cuando los datos no son suficientes para ser divididos en dos grupos: entrenamiento y validación, aún es posible usar en lo modelos de regresión las técnicas basadas en  la distribución predictiva como se decribe en esta lección. Primero, recordemos como generar nuevas observaciones con la distribución predictiva.

# Predición de  datos no observados

Consideremos un modelo bayesiano con parámetros $\boldsymbol{\theta}$ y datos continuos $\mathbf{y}$. Vamos a considerar la posterior $p(\boldsymbol{\theta}|y)$ . Entonces generamos nuevos datos como sigue:

1. Generamos una muestra $\boldsymbol{\theta}^{(1)}, \ldots, \boldsymbol{\theta}^{(n)}$, a partir de la posterior $p(\boldsymbol{\theta}|y)$.

2. Para cada muestra $\boldsymbol{\theta}^{(k)}$ se obtiene una muestra $\tilde{y}_k$ a partir de valores de un vector concido de variables predictoras, digamos $\mathbf{z}$,  y $\boldsymbol{\theta}^{(k)}$. La muestra se obtiene  de la verosimilitud $p(y|\boldsymbol{\theta}^{(k)},\mathbf{z})$.



## Ejemplo. Modelo de Regresión Normal homocedástico $\mathcal{N}(\mu_i,\sigma^2)$

Si este es el modelo verdadero que produjo los datos, lo que ocurrió es que 

### Generación de los datos

Este es el proceso que teóricamente ocurrió para generar los datos.

1. $\boldsymbol{\theta}$ fué generado a partir de una distribución (desconocido) $P(\boldsymbol{\theta})$. En este ejemplo se tiene que  $\boldsymbol{\theta} = (\alpha,\boldsymbol{\beta},\sigma)'$.

2. Cada observación $y_i$ tienen distribución $\mathcal{N}(\mu_i,\sigma^2)$, en donde $\mu_i = \alpha + \mathbf{x}_i'\boldsymbol{\beta}$. Así que el dato observado $y_i$ es obtenido como una muestra aleatoria de $\mathcal{N}(\mu_i,\sigma^2)$. 


### Generación de nuevos datos

Nosotros construimos la posterior $p(\boldsymbol{\theta}|y)$. Esta es nuestra aproximación de la distribución desconocida $P(\boldsymbol{\theta})$. Entonces los datos simulados no observadosse obtienen como sigue.

1. Se obtiene una muestra $\boldsymbol{\theta}^{(1)}, \ldots, \boldsymbol{\theta}^{(n)}$, a partir de la posterior $p(\boldsymbol{\theta}|y)$.  Aquí $\boldsymbol{\theta}^{(k)} =(\alpha^{(k)}, \boldsymbol{\beta}^{(k)}, \sigma^{(k)})'$.

2. A partir de cada muestra posterior $(\alpha^{(k)}, \boldsymbol{\beta}^{(k)}, \sigma^{(k)})'$ se calcula $\mu_z^{(k)}= \alpha^{(k)} +\mathbf{z}'\boldsymbol{\beta}^{(k)} $. El dato no observado $\tilde{y}$ en este caso se obtiene como una muestra de la distribución $\mathcal{N}(\mu_z^{(k)},  \sigma^{(k)})$.

# Generación de datos para validación cruzada en modelos de regresión (dejando uno por fuera)

Esta  técnica es conocida como *loo* del inglés leave one out, o dejar uno por fuera.

En este caso, el procedimiento a seguir es el siguiente. Sea $\mathbf{y}_{-i}$ el conjunto de observaciones excepto $y_i$. Sea $\mathbf{x}_{i}$  el vector de varables predictoras asociadas a $y_i$. Entonces, una réplica de la observación $y_i$ se obtiene como sigue.

1. Se obtiene una muestra $\boldsymbol{\theta}^*_{-i}$ de la posterior $p(\boldsymbol{\theta}|\mathbf{y}_{-i})$.
2. Se obtiene una réplica de  $y_i$ como una muestra  de la verosimilitud $p(y|\boldsymbol{\theta}^*_{-i},\mathbf{x}_{i})$.

## Validación cruzada

Lo que se hace es generar muestras *loo*. Sea  $\boldsymbol{\theta}^*_{-i}$ una muestra de la posterior $p(\boldsymbol{\theta}|\mathbf{y}_{-i})$. Definimos $\mu_{-i}$ como la media aosciada al problema, entonces



### Residuales de Pearson estandarizados loo

$$
\begin{align}
r_{loo}(i) &= \frac{(y_i-\mu_{-i})}{\sigma_{-i}}\\
T(\mathbf{y}|\boldsymbol{\theta}_{-i}) &= \sum_{i=1}^n \frac{(y_i-\mu_{-i})}{\sigma_{-i}}
\end{align}
$$

###   Discrepancia Deviance 

$$
\begin{align}
T(y_i|\boldsymbol{\theta}_{-i}) &= -2 \log f(y_i|\boldsymbol{\theta}_{-i})\\ 
T(\mathbf{y}|\boldsymbol{\theta}_{-i})&=-2 \sum_{i=1}^n \log f(y_i|\boldsymbol{\theta}_{-i})
\end{align}
$$



# Detección de valores extremos (outliers)

El p-valor Bayesiano puede usarse para detectar valores extremos de los datos, de acuerdo con la distribución del problema.

La idea clave es la siguiente. Defina un unmbral, digamos $\text{umbral} = 2.5$a  partir del cual, si se tiene una variable estandarizada, un valor mayor a este umbral puede considerarse sospechoso de ser un valor extremo.

1. En cada paso del algoritmo de muestreo calcule el residual. es decir
$$
r_i^{(k)} = \frac{y_i-\mu_{-i}^{(k)}}{\sigma_{-i}^{(k)}}
$$
2. Calcule la estadística $1_{\{|r_i^{(k)}|>\text{umbral}\}}$
3. Si se obtienen $m$ muestras, el p-valor Bayesiano es dado por

$$
p_{\text{outlier}}(y_i) = \frac{1}{m}\sum_{k=1}^m 1_{\{|r_i^{(k)}|>\text{umbral}\}}
$$
 
Entonces, valores grandes de $p_{\text{outlier}}(y_i)$ señalan a $y_i$ como un valor extremo, con una probabilidad dada por este p-valor Bayesiano.

# Implementación en Stan

En esta sección implementamos los conceptos de la lección para los datos stack_loss de lecciones previas. Usaremos el modelo normal.

In [1]:
# datos
import numpy as np
p = 3
N = 1
umbral = 2.5
y = np.array([43, 37, 37, 28, 18, 18, 19, 20, 15, 14, 14, 13, 11, 12, 8, 
7, 8, 8, 9, 15, 15])
x = np.resize((80, 80, 75, 62, 62, 62, 62, 62, 59, 58, 58, 58, 58, 
58, 50, 50, 50, 50, 50, 56, 70, 27, 27, 25, 24, 22, 23, 24, 24, 
23, 18, 18, 17, 18, 19, 18, 18, 19, 19, 20, 20, 20, 89, 88, 90, 
87, 87, 87, 93, 93, 87, 80, 89, 88, 82, 93, 89, 86, 72, 79, 80, 
82, 91), (21, 3))




In [17]:
# leave ou observation i

i = 0
y_data = np.hstack([y[:i],y[(i+1):]])
y_out = y[i]
x_data = np.delete(x,i,axis=0)
x_out = x[i,:]
x_out

stacks_data = {'p': p, 'N': N, 'umbral':umbral, 
               'y': y_data,
               'x': x_data,
               'y_out',y_out,
               'x_out':x_out,}

array([80, 80, 75])

In [7]:
import numpy as np
import pystan
%load_ext stanmagic
%matplotlib inline
import numpy as np
import pylab as pl

The stanmagic extension is already loaded. To reload it, use:
  %reload_ext stanmagic


In [20]:
%%stan -f normal_stack_loss_reg-p_val.stan
// Linear Model with normal Errors

data {
  int<lower=0> N;  // number of observations
  int<lower=0> p;  // number of explained variables
  vector[N] y;     // observations
  matrix[N,p] x;   // design matrix
  real umbral;     // threshold for outlier detection
  real y_out;      // y loo
  vector[p] x_out;      // x loo
} 



transformed data {
  matrix[N,p] z;
  vector[p] mean_x;
  vector[p] sd_x;
  real y_min;
  real y_max;
  # order statistics
  y_min = min(y);
  y_max = max(y);
  // standardize the x's 
  for (j in 1:p) { 
    mean_x[j] = mean(col(x,j)); 
    sd_x[j] = sd(col(x,j)); 
    for (i in 1:N)  
      z[i,j] = (x[i,j] - mean_x[j]) / sd_x[j]; 
  }
}

parameters {
  real beta_0; 
  vector[p] beta; 
  real<lower=0> sigma; 
} 

transformed parameters {
    vector[N] mu;
    mu = beta_0 + z*beta;
}

model {
  beta_0 ~ normal(0, 5); 
  beta   ~ normal(0, 5);
  sigma  ~ cauchy(0, 2);
  y      ~ normal(mu, sigma);
} 

generated quantities {
  vector[N] residual; // Pearson residual
  vector[N] outlier;  // test for outlier
  vector[N] resi_dev; // residual deviance 
  vector[N] y_rep;    // replica
  real pval_model;    // p-value model
  real pval_min;      // p-value minimum
  real pval_max;      // p-value maximum
  vector[N] Ty;       // Discrepance (data)
  vector[N] Ty_rep;   // Discrepance (rep)
    
  for (i in 1:N){
    residual[i] = (y[i] - mu[i]) / sigma;
    outlier[i]  = step(residual[i] - umbral);
    // cambiar la siguiente línea en cada caso
    resi_dev[i] = (y[i] - mu[i])/sigma;
    // cambiar la siguientes líneas en cada caso
    y_rep[i]  = normal_rng(mu[i],sigma);
    Ty[i]     =   normal_lpdf(y[i]|mu[i],sigma);
    Ty_rep[i] =   normal_lpdf(y_rep[i]|mu[i],sigma);  
 }
 // cambiar la siguiente línea en cada caso
 pval_model = step(sum(Ty)-sum(Ty_rep));
 pval_min   = step(y_min - min(y_rep));
 pval_max   = step(y_max - max(y_rep));
}    

Using pystan.stanc compiler..
-------------------------------------------------------------------------------
Model compiled successfully. Output stored in _stan_model object.
Type _stan_model in a cell to see a nicely formatted code output in a notebook
     ^^^^^^^^^^^
Access model compile output properties
_stan_model.model_file -> Name of stan_file [normal_stack_loss_reg-p_val.stan]
_stan_model.model_name -> Name of stan model [normal_stack_loss_reg-p_val_model]
_stan_model.model_code -> Model code [// Linear Model with ....]


In [21]:
_stan_model

model_file: normal_stack_loss_reg-p_val.stan
model_name: normal_stack_loss_reg-p_val_model
model_code:
// Linear Model with normal Errors

data {
  int<lower=0> N;  // number of observations
  int<lower=0> p;  // number of explained variables
  vector[N] y;     // observations
  matrix[N,p] x;   // design matrix
  real umbral;     // threshold for outlier detection
} 



transformed data {
  matrix[N,p] z;
  vector[p] mean_x;
  vector[p] sd_x;
  real y_min;
  real y_max;
  # order statistics
  y_min = min(y);
  y_max = max(y);
  // standardize the x's 
  for (j in 1:p) { 
    mean_x[j] = mean(col(x,j)); 
    sd_x[j] = sd(col(x,j)); 
    for (i in 1:N)  
      z[i,j] = (x[i,j] - mean_x[j]) / sd_x[j]; 
  }
}

parameters {
  real beta_0; 
  vector[p] beta; 
  real<lower=0> sigma; 
} 

transformed parameters {
    vector[N] mu;
    mu = beta_0 + z*beta;
}

model {
  beta_0 ~ normal(0, 5); 
  beta   ~ normal(0, 5);
  sigma  ~ cauchy(0, 2);
  y      ~ normal(mu, sigma);
} 

generated quantities {
  vector[N] residual; // Pearson residual
  vector[N] outlier;  // test for outlier
  vector[N] resi_dev; // residual deviance 
  vector[N] y_rep;    // replica
  real pval_model;    // p-value model
  real pval_min;      // p-value minimum
  real pval_max;      // p-value maximum
  vector[N] Ty;       // Discrepance (data)
  vector[N] Ty_rep;   // Discrepance (rep)
    
  for (i in 1:N){
    residual[i] = (y[i] - mu[i]) / sigma;
    outlier[i]  = step(residual[i] - umbral);
    // cambiar la siguiente línea en cada caso
    resi_dev[i] = (y[i] - mu[i])/sigma;
    // cambiar la siguientes líneas en cada caso
    y_rep[i]  = normal_rng(mu[i],sigma);
    Ty[i]     =   normal_lpdf(y[i]|mu[i],sigma);
    Ty_rep[i] =   normal_lpdf(y_rep[i]|mu[i],sigma);  
 }
 // cambiar la siguiente línea en cada caso
 pval_model = step(sum(Ty)-sum(Ty_rep));
 pval_min   = step(y_min - min(y_rep));
 pval_max   = step(y_max - max(y_rep));
}

In [22]:
normal_stack_loss_reg = pystan.StanModel(file=_stan_model.model_file)

INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_721bdac6da1bb4db402b057e8b2b0723 NOW.


In [27]:
# Muestreo
np.random.seed(500)
normal_stack_loss_reg_sample = normal_stack_loss_reg.sampling(data=stacks_data, iter =4000)

In [28]:
print(normal_stack_loss_reg_sample)

Inference for Stan model: anon_model_721bdac6da1bb4db402b057e8b2b0723.
4 chains, each with iter=4000; warmup=2000; thin=1; 
post-warmup draws per chain=2000, total post-warmup draws=8000.

               mean se_mean     sd   2.5%    25%    50%    75%  97.5%  n_eff   Rhat
beta_0        14.17    0.03   2.45   8.97  12.66   14.3  15.83  18.62   5905    1.0
beta[1]        0.92    0.05   3.96  -6.93  -1.78   0.93   3.59   8.64   6056    1.0
beta[2]       -0.03    0.05   4.13  -8.21  -2.82  -0.05   2.79   7.88   6261    1.0
beta[3]       -0.87    0.05   4.03  -8.89  -3.54  -0.88   1.88    6.9   6113    1.0
sigma         11.28    0.03   2.04   8.12   9.84  10.99  12.38  16.17   5116    1.0
mu[1]         14.38    0.04   3.26   7.52  12.32   14.5  16.55  20.39   6625    1.0
mu[2]          14.2    0.03   2.51   8.96  12.67  14.33  15.91  18.81   5946    1.0
mu[3]         14.29    0.03   2.55   8.92  12.72  14.41  16.02  18.93   5783    1.0
mu[4]         14.19    0.03   2.46   8.99  12.69  14.31

In [ ]:
normal_stack_loss_reg_sample

# Tarea

1. Repita el ejercicio para las distribuciones Laplace y $t_4$
2. ¿Cuáles datos se reportarían como outlier? Distuca su respuesta.
3. ¿Cuál model es aparentemente mejor?